In [1]:
import os
os.environ['DT'] = '.5'
os.environ['NX'] = '32'
os.environ['N_STEPS'] = '16'
os.environ['N_ITERS'] = '3'
os.environ['N_REPEATS'] = '5'
os.environ['MAX_THREADS'] = '3'

In [2]:
import subprocess
import numpy as np
import sys
import json

In [3]:
data_libmpdata = {}
for n_thread in range(1, int(os.environ['MAX_THREADS'])+1):
    value = np.inf
    for _ in range(int(os.environ['N_REPEATS'])):
        info = subprocess.run(
            ["build/revolving_sphere_3d"],
            capture_output=True,
            env={**os.environ, "OMP_NUM_THREADS": str(n_thread)}
        )
        if info.returncode != 0:
            print(info)
            assert False
        value = min(value, float(str(info.stderr).split(" ")[3][:-1]))
    data_libmpdata[str(n_thread)] = value
print(data_libmpdata)

{'1': 0.125215, '2': 0.0639015, '3': 0.0436632}


In [ ]:
for ndj in ('0', '1'):
    info = subprocess.run(
        [sys.executable, "revolving_sphere_3d.py"],
        capture_output=True,
        env={**os.environ, 'NUMBA_DISABLE_JIT': ndj}
    )
    if info.returncode != 0:
        print(info)
        assert False
data_pympdata = {}
for ndj in ('0', '1'):
    with open(f'data_NDJ={ndj}.json', 'r') as file:
        for k, v in json.load(file).items():
            data_pympdata[k] = v
print(data_pympdata)

In [ ]:
from matplotlib import pyplot
from atmos_cloud_sim_uj_utils import show_plot

fig, (ax1, ax2) = pyplot.subplots(2, 1, sharex=True)            
handles = []

for grid, marker in {'static': 'p', 'dynamic': '+'}.items():
    handles.append(ax2.plot(
        data_pympdata[grid].keys(), data_pympdata[grid].values(),
        label=f'PyMPDATA: {grid} grid',
        linewidth=2.5, marker=marker
    )[0])
handles.append(ax2.plot(
    data_libmpdata.keys(), data_libmpdata.values(),
    label="libmpdata++",
    linewidth=2.5, color='green', marker='o'
)[0])

x = data_pympdata['static'].keys()
handles.append(ax1.plot(
    x, [data_pympdata['nojit']]*len(x),
    label='PyMPDATA: Numba JIT disabled',
    linewidth=2.5, color='red'
)[0])

ax1.spines['bottom'].set_visible(False)
ax2.spines['top'].set_visible(False)
ax1.xaxis.tick_top()
ax1.tick_params(labeltop=False)
ax2.xaxis.tick_bottom()

d = .0075  # how big to make the diagonal lines in axes coordinates
# arguments to pass to plot, just so we don't keep repeating them
kwargs = dict(transform=ax1.transAxes, color='k', clip_on=False)
ax1.plot((-d, +d), (-d, +d), **kwargs)        # top-left diagonal
ax1.plot((1 - d, 1 + d), (-d, +d), **kwargs)  # top-right diagonal

kwargs.update(transform=ax2.transAxes)  # switch to the bottom axes
ax2.plot((-d, +d), (1 - d, 1 + d), **kwargs)  # bottom-left diagonal
ax2.plot((1 - d, 1 + d), (1 - d, 1 + d), **kwargs)  # bottom-right diagonal

for ax in (ax1, ax2):
    ax.grid()
    ax.set_yscale('log')

ax1.legend(handles=handles[-1::-1], loc='lower right')

pyplot.xlabel("number of threads")
pyplot.ylabel("                                      wall time [s]\n")
show_plot('fig_X.pdf')